In [1]:
import numpy as np
import pandas as pd

In [28]:
def createuber():
    #this function reads in uber.csv and returns the data in pd.DataFrame format
    uber = pd.read_csv('uber.csv')
    uber.pickup_time = pd.to_datetime(uber.pickup_time)
    return uber

In [29]:
def createtaxi():
    #this function reads in yellow_trip_{year}-{month}.csv and returns the data in pd.DataFrame format
    filenames = []
    for i in range(2009,2016):
        for j in range(1,13):
            if j <10:
                filename = f'yellow_tripdata_{i}-0{j}.csv'
                filenames.append(filename)
            else:
                filename = f'yellow_tripdata_{i}-{j}.csv'
                filenames.append(filename)
    taxi = pd.read_csv('yellow_tripdata_2009-01.csv',usecols = ['pickup_time','dropoff_time','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','charge','tip'],nrows = 2381)
    for i in range(1,len(filenames)):
        df = pd.read_csv(filenames[i],usecols = ['pickup_time','dropoff_time','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','charge','tip'], nrows = 2381)
        taxi = pd.concat([taxi,df])
    taxi.reset_index(inplace = True, drop = True)
    taxi.pickup_time = pd.to_datetime(taxi.pickup_time)
    taxi.dropoff_time = pd.to_datetime(taxi.dropoff_time)
    return taxi

In [30]:
def createhourlyweather():
    #this function reads in {year}_weather_hours.csv and returns the data in pd.DataFrame format
    filenames = []
    for i in range(2009,2016):
        filenames.append(f'{i}_weather_hours.csv')
    weather_hourly = pd.read_csv('2009_weather_hours.csv')
    for i in range(1,len(filenames)):
        df = pd.read_csv(filenames[i])
        weather_hourly = pd.concat([weather_hourly,df])
    weather_hourly.reset_index(inplace = True, drop = True)
    weather_hourly.DATE = pd.to_datetime(weather_hourly.DATE)
    weather_hourly['DATE'] = weather_hourly['DATE'].dt.floor('H')
    weather_hourly = weather_hourly.drop_duplicates('DATE',keep = 'first')
    return weather_hourly
    

In [31]:
def createdailyweather():
    #this function reads in {year}_weather_days.csv and returns the data in pd.DataFrame format
    filenames = []
    for i in range(2009,2016):
        filenames.append(f'{i}_weather_days.csv')
    weather_daily = pd.read_csv('2009_weather_days.csv')
    for i in range(1,len(filenames)):
        df = pd.read_csv(filenames[i])
        weather_daily = pd.concat([weather_daily,df])
    weather_daily.reset_index(inplace = True, drop = True)
    weather_daily.DATE = pd.to_datetime(weather_daily.DATE)
    return weather_daily

In [32]:
uber = createuber()
taxi = createtaxi()
weather_hourly= createhourlyweather()
weather_daily = createdailyweather()